In [1]:
import os
import sys
import json
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from typing import List, Tuple, Dict, Any, Optional

In [ ]:
PATH_ROOT_DIR: str = ".."

INFO_TO_USE_FOR_CLUSTERING: List[str] = [
    'exception_message', 'essential_file_content']

PCA_COMPONENTS: int = 3

AI_MODEL: str = "llama-3.1-70b-versatile"  # or "gpt-4o-mini"

N_POSITIVE_EXAMPLES_TO_SHOW_LLM: int = 5

# Test run
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__14_48__qiskit")
# Run with 10k programs: v001.yaml
# program_bank/v001/2024_10_04__15_10__qiskit
# DIR_PROGRAM_BANK = os.path.join("..", "program_bank", "v001", "2024_10_04__15_10__qiskit")
# program_bank/v002/2024_10_07__17_03__qiskit
# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__qiskit")

# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v004", "2024_10_18__16_09__sample_errors")


# DIR_PROGRAM_BANK: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__qiskit")
# DIR_SAMPLE_ERRORS: str = os.path.join(
#     "..", "program_bank", "v005", "2024_10_22__14_17__sample_errors")

# inspected folders
# program_bank/v005/2024_10_22__22_32__qiskit
# program_bank/v007/2024_10_31__17_51__qiskit
# program_bank/v008/2024_11_07__15_05__qiskit
DIR_PROGRAM_BANK: str = os.path.join(
    "..", "program_bank", "v008", "2024_11_07__15_05__qiskit")
DIR_SAMPLE_ERRORS: str = os.path.join(
    "..", "program_bank", "v008",
    "2024_11_07__15_05__sample_errors_no_pennylane")


PROMPT_CONTENT: str = "BOTH_ERROR_AND_PROGRAM"

In [29]:
# print the current folder
print(f"Current folder: {os.getcwd()}")

all_global_vars = dict(globals().copy())
# print all global variables
for k, v in all_global_vars.items():
    if k.isupper():
        print(f"{k}: {v}")

Current folder: /home/paltenmo/projects/crossplatform/notebooks
PATH_ROOT_DIR: ..
INFO_TO_USE_FOR_CLUSTERING: ['exception_message', 'essential_file_content']
PCA_COMPONENTS: 3
AI_MODEL: llama-3.1-70b-versatile
N_POSITIVE_EXAMPLES_TO_SHOW_LLM: 5
DIR_PROGRAM_BANK: ../program_bank/v008/2024_11_07__15_05__qiskit
DIR_SAMPLE_ERRORS: ../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane
PROMPT_CONTENT: BOTH_ERROR_AND_PROGRAM
SAMPLE_SIZE: 9


In [30]:
import openai
import groq
PATH_GROQ_TOKEN = os.path.join(PATH_ROOT_DIR, "groq_token.txt")
GROQ_TOKEN = Path(PATH_GROQ_TOKEN).read_text().strip()

# Export the GROQ_TOKEN as GROQ_API_KEY
os.environ['GROQ_API_KEY'] = GROQ_TOKEN

PATH_OPENAI_API_KEY = os.path.join(PATH_ROOT_DIR, "openai_token.txt")
OPENAI_API_KEY = Path(PATH_OPENAI_API_KEY).read_text().strip()

os.environ['OPENAI_API_KEY'] = OPENAI_API_KEY

In [31]:
# count generated programs in the program bank
count_generated_programs = len(list(Path(DIR_PROGRAM_BANK).rglob("*.py")))
print(f"Number of generated programs: {count_generated_programs}")

# count the number of errors
count_errors = len(list(Path(DIR_PROGRAM_BANK).rglob("*_error.json")))
print(f"Number of errors: {count_errors}")

Number of generated programs: 10000
Number of errors: 19393


## Read Files

In [36]:
from multiprocessing import Pool
import re


def read_json_file(file_path: str) -> Dict[str, Any]:
    with open(file_path, 'r') as file:
        data = json.load(file)
        data["error_file"] = os.path.basename(file_path)
        return data


def read_all_programs(dir_path: str) -> pd.DataFrame:
    json_files = [os.path.join(dir_path, f) for f in os.listdir(
        dir_path) if re.match(r'.*_error\.json$', f)]
    records = []

    with Pool() as pool:
        results = pool.map(read_json_file, json_files)
        records.extend(results)

    return pd.DataFrame(records)


# Use the function to read all programs
df_errors = read_all_programs(DIR_PROGRAM_BANK)

# augment the dataframe with the proram content


def read_file_content(file_name: str) -> str:
    file_path = os.path.join(DIR_PROGRAM_BANK, file_name)
    with open(file_path, 'r') as file:
        return file.read()


# Use parallel apply to read the file content
with Pool() as pool:
    df_errors['file_content'] = pool.map(
        read_file_content, df_errors['current_file'])

# Alternatively, you can use apply without parallel processing
# df_errors['file_content'] = df_errors['current_file'].apply(read_file_content)

df_errors.head()
n_errors = len(df_errors)
print(f"Number of errors: {n_errors}")

Number of errors: 19393


# Exclude Pennylane (errors)

In [37]:
df_errors['involved_functions'] = df_errors['involved_functions'].astype(
    str)
df_errors = df_errors[
    ~df_errors['involved_functions'].str.contains('pennylane')]
n_errors = len(df_errors)
print(f"Number of errors without PennyLane: {n_errors}")

Number of errors without PennyLane: 12468


# Minimize Sample

In [39]:
import shutil

# keep only errors with `The circuits are not equivalent` in the exception message
df_errors = df_errors[df_errors['exception_message'].str.contains(
    'The circuits are not equivalent')]


SAMPLE_SIZE = 10
actual_sample_size = min(SAMPLE_SIZE, len(df_errors))


# pick a sample of error files
sample_errors = df_errors.sample(actual_sample_size, random_state=42)

# get the paths of the error files
sample_errors_paths = [os.path.join(
    DIR_PROGRAM_BANK, f) for f in sample_errors['error_file']]


def get_corresponding_program_file(path_error_file: str) -> str:
    # remove the fixed length prefix "_ba6b02_error.json"
    return path_error_file[:-18] + ".py"


# copy them to a new folder
os.makedirs(DIR_SAMPLE_ERRORS, exist_ok=True)
for file_path in sample_errors_paths:
    shutil.copy(file_path, DIR_SAMPLE_ERRORS)
    print(f"Copying {file_path} to {DIR_SAMPLE_ERRORS}")
    corresponding_program_file = get_corresponding_program_file(file_path)
    shutil.copy(corresponding_program_file, DIR_SAMPLE_ERRORS)
    print(f"Copying {corresponding_program_file} to {DIR_SAMPLE_ERRORS}")

Copying ../program_bank/v008/2024_11_07__15_05__qiskit/qiskit_circuit_5q_10g_7916_b68352_1b0650_error.json to ../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane
Copying ../program_bank/v008/2024_11_07__15_05__qiskit/qiskit_circuit_5q_10g_7916_b68352.py to ../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane
Copying ../program_bank/v008/2024_11_07__15_05__qiskit/qiskit_circuit_5q_10g_4250_3f1a07_dc1379_error.json to ../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane
Copying ../program_bank/v008/2024_11_07__15_05__qiskit/qiskit_circuit_5q_10g_4250_3f1a07.py to ../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane
Copying ../program_bank/v008/2024_11_07__15_05__qiskit/qiskit_circuit_5q_10g_953_0914f5_f6d04b_error.json to ../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane
Copying ../program_bank/v008/2024_11_07__15_05__qiskit/qiskit_circuit_5q_10g_953_0914f5.py to ../program_bank/v008/2024_11_07__15_05__sample_error

In [40]:
# get the content of their error messages
sample_errors['exception_message']

16161    The circuits are not equivalent: /workspace/qi...
3579     The circuits are not equivalent: /workspace/qi...
12373    The circuits are not equivalent: /workspace/qi...
2129     The circuits are not equivalent: /workspace/qi...
19209    The circuits are not equivalent: /workspace/qi...
4611     The circuits are not equivalent: /workspace/qi...
5813     The circuits are not equivalent: /workspace/qi...
5214     The circuits are not equivalent: /workspace/qi...
14780    The circuits are not equivalent: /workspace/qi...
Name: exception_message, dtype: object

In [41]:
import sys
sys.path.append("..")

## Triage All Files 
Including:
1. minimization
2. rerunning the code
3. running the analysis notebook

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [44]:
from analysis_and_reporting.triage import analyze_and_report

error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, f)
    for f in os.listdir(DIR_SAMPLE_ERRORS)
    if re.match(r'.*_error\.json$', f)
]

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)

# python -m analysis_and_reporting.triage --path_error_json program_bank/v007/2024_10_31__17_51__qiskit/qiskit_circuit_5q_10g_9_160f04_cfe9f1_error.json --parent_report_folder reports/v008 --clue_message 'not equivalent'


# def analyze_and_report(
#         path_error_json: str, path_program: str, parent_report_folder: str,
#         clue_message: str, analysis_notebook: str)

for error_path in error_paths_in_sample_folder:
    try:
        analyze_and_report(
            path_error_json=error_path,
            path_program=get_corresponding_program_file(error_path),
            parent_report_folder=os.path.join("..", "reports", "v008"),
            clue_message='not equivalent',
            analysis_notebook='010_Triage_Not_Equivalence.ipynb'
        )
    except Exception as e:
        print(f"Error: {e}")
        print("Skipping to the next error file")

Error paths in sample folder (9):
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_9196_8b67cd_55aec5_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_4250_3f1a07_dc1379_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_7738_eff6c4_763398_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_4730_832a71_782f83_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_8118_fb9caa_2d3395_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_953_0914f5_f6d04b_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_7434_c7a9b4_11c519_error.json
../program_bank/v008/2024_11_07__15_05__sample_errors_no_pennylane/qiskit_circuit_5q_10g_7916_b68352_1b0650_error.json
../program_bank

[14:29:25] Involved functions:                                                             ]8;id=721102;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=719236;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=208928;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=592457;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=146285;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=969348;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_qiskit                                                  ]8;id=111536;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=233061;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

...

[14:30:37] Involved functions:                                                             ]8;id=779209;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=127047;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=787327;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=21641;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=98622;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=898023;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_qiskit                                                  ]8;id=511589;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=782588;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

Error parsing: Command 'python /workspace/to_execute.py' in image 'qiskit_runner' returned non-zero exit status 1: b'Traceback (most recent call last):\n  File "/workspace/to_execute.py", line 105, in <module>\n    target_qc = qc\nNameError: name \'qc\' is not defined\n'
Failed to execute to_execute.py in Docker.
Output: Saved the Pytket circuit to /workspace/to_execute_qc_pytket.qasm
Saved the Qiskit circuit to /workspace/to_execute_qc_qiskit.qasm
Exported QASM files: [('/workspace/to_execute_qc_pytket.qasm', 'export_to_qasm_with_pytket'), ('/workspace/to_execute_qc_qiskit.qasm', 'export_to_qasm_with_qiskit')]
The circuits are equivalent: /workspace/to_execute_qc_pytket.qasm, /workspace/to_execute_qc_qiskit.qasm

Program to_execute.py executed successfully.
Output: Saved the Pytket circuit to /workspace/to_execute_qc_pytket.qasm
Saved the Qiskit circuit to /workspace/to_execute_qc_qiskit.qasm
Exported QASM files: [('/workspace/to_execute_qc_pytket.qasm', 'export_to_qasm_with_pytket'),

## Minimize All Files

In [11]:
from analysis_and_reporting.ddmin_target_file import minimize_instructions
from pathlib import Path

error_paths_in_sample_folder = [
    os.path.join(DIR_SAMPLE_ERRORS, f)
    for f in os.listdir(DIR_SAMPLE_ERRORS)
    if re.match(r'.*_error\.json$', f)
]

# run minimization command:
# python -m analysis_and_reporting.ddmin_target_file --input_folder {folder_with_files} --path_to_error {path_to_json_file} --clue_message 'The circuits are not equivalent'

n_error_files = len(error_paths_in_sample_folder)
print(f"Error paths in sample folder ({n_error_files}):")
for error_path in error_paths_in_sample_folder:
    print(error_path)

# add the parent folder to the path
sys.path.append("..")

for error_path in error_paths_in_sample_folder:
    path_minimized_file = error_path.replace("_error.json", "_error_min.py")
    # check if the same file exists in the sample folder with extension _min.py
    # if yes then skip
    if os.path.exists(path_minimized_file):
        print(f"Skipping minimization for {error_path}")
        continue

    print(f"Running minimization for {error_path}")
    minimize_instructions(
        input_folder=Path(DIR_PROGRAM_BANK),
        path_to_error=Path(error_path),
        clue_message='The circuits are not equivalent'
    )

    # copy the minimized file from DIR_PROGRAM_BANK to the sample folder
    name_minimized_file = os.path.basename(path_minimized_file)
    shutil.copy(
        os.path.join(DIR_PROGRAM_BANK, name_minimized_file),
        path_minimized_file
    )

Error paths in sample folder (10):
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_35328_d5b463_9d5e4d_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_40142_843f1f_c4b462_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_33594_363d0d_737d3d_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_24368_522f46_c267bd_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_62099_fd6240_ba6b02_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_16712_52499e_26f332_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_7831_c7cfba_51499c_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_64167_2a247b_08cf7c_error.json
../program_bank/v007/2024_10_31__17_51__sample_errors/qiskit_circuit_5q_10g_30841_b17cde_e35c28_error.json
../

[15:26:15] Involved functions:                                                             ]8;id=502748;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=152212;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#206\206]8;;\

             - compare_qasm_via_qcec                                                       ]8;id=617254;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=869869;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_bqskit                                                  ]8;id=963044;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=214076;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=629891;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=477492;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

...

             - compare_qasm_via_qcec                                                       ]8;id=122624;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=124856;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pennylane                                               ]8;id=528415;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=141979;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

             - export_to_qasm_with_pytket                                                  ]8;id=864710;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=807630;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#208\208]8;;\

Error parsing: Command 'python /workspace/to_execute.py' in image 'qiskit_runner' returned non-zero exit status 1: b'Traceback (most recent call last):\n  File "/workspace/to_execute.py", line 123, in <module>\n    target_qc = qc\nNameError: name \'qc\' is not defined\n'
Failed to execute to_execute.py in Docker.
Output: Saved the PennyLane circuit to /workspace/to_execute_qc_pennylane.qasm
Saved the Pytket circuit to /workspace/to_execute_qc_pytket.qasm
Exported QASM files: [('/workspace/to_execute_qc_pennylane.qasm', 'export_to_qasm_with_pennylane'), ('/workspace/to_execute_qc_pytket.qasm', 'export_to_qasm_with_pytket')]
Log has been saved to /workspace/to_execute_8a3b7b_error.json

Program to_execute.py executed successfully.
New exception message: The circuits are not equivalent: /workspace/to_execute_qc_pennylane.qasm, /workspace/to_execute_qc_pytket.qasm
Error parsing: Command 'python /workspace/to_execute.py' in image 'qiskit_runner' returned non-zero exit status 1: b'  File "/w

[15:41:16] Minimized file saved as:                                                        ]8;id=865393;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py\ddmin_target_file.py]8;;\:]8;id=850336;file:///home/paltenmo/projects/crossplatform/analysis_and_reporting/ddmin_target_file.py#240\240]8;;\
           ../program_bank/v007/2024_10_31__17_51__qiskit/qiskit_circuit_5q_10g_32186_4678                         
           78_3b2333_error_min.py                                                                                  